# Implementación de PCA en NumPy

## Objetivos
* Implementación de PCA en NumPy paso a paso
* Comparación de resultados con Scikit-learn

In [3]:
import numpy as np

## Implementación

1. Dado un dataset $X \in \mathbb{R}^{n, d}$, con $n$ muestras y $d$ features, queremos reducir sus dimensiones a $m$. Para ello, el primer paso es centrar el dataset (Hint: usen np.mean)

In [3]:
def cent_dataset(x):
    """
    Esta funcion recibe un dataset y lo centra con el fin de que tenga media 0
    Devuelve el data set centrado y la matriz con los valores de la media de cada uno
    """
    media = x.mean(axis=1)
    return x - media[:,None] , media

2. Obtener la matriz de covarianza de $X^T$, revisar en la teoría por qué utilizamos la transpuesta. Buscar en la documentación de NumPy qué funciones se pueden utilizar.

In [28]:
def transpose_dataset(x):
    return x.transpose()

3. Calcular los autovalores y autovectores de la matriz de covarianza. Revisar la documentación de NumPy.

In [7]:
def covarianza_dataset(x):
    s = x.dot(x.transpose()) / x.shape[0]
    autovalores, autovectores = np.linalg.eigh(s)
        # np.linalg.eigh():
        # Return the eigenvalues and eigenvectors of a complex Hermitian
            #(conjugate symmetric) or a real symmetric matrix.
        # Returns two objects, a 1-D array containing the eigenvalues of a, 
            # and a 2-D square array or matrix (depending on the input type) 
            # of the corresponding eigenvectors (in columns).
    return s, autovalores, autovectores

4. Ordernar los autovectores en el sentido de los autovalores decrecientes, revisar la teoría de ser necesario.

In [8]:
def sort_eig(autovalores, autovectores):
    arg_sort = np.argsort(autovalores)[::-1]
    autovalores = autovalores[arg_sort]
    autovectores = autovectores[:,arg_sort]
    return autovalores, autovectores

5. Proyectar el dataset centrado sobre los $m$ autovectores más relevantes (Hint: usen np.dot).

In [38]:
def calculo_B(m, autovectores):
    """m debe ser menor o igual al numero de columanas de la matriz de autovectores"""
    return autovectores[:,:m]

6. Consolidar los pasos anteriores en una función o clase PCA.

In [36]:
def PCA_numpy(x, n_components=2):
    # centro el dataset
    x_cent = x.copy()
    x_cent = x_cent -  x.mean(axis=0)
    
    s = (x_cent.T@x_cent) / x_cent.shape[0]
    autovalores, autovectores = np.linalg.eig(s)
    
    arg_sort = np.argsort(autovalores)[::-1]
    autovalores = autovalores[arg_sort]
    autovectores = autovectores[:,arg_sort]
    
    B = autovectores[:,:n_components]
    
    z = np.dot(x_cent,B)
    x_recuperada = np.dot(z,B.T) + x.mean(axis=0)

    return B, z, x_recuperada

7. Comparar los resultados obtenidos con el modelo de PCA implementado en Scikit-learn ([ver documentación](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)). Tomar como dataset:

$X=\begin{bmatrix}
0.8 & 0.7\\
0.1 & -0.1
\end{bmatrix}$

Se debe reducir a un componente. Verificar los resultados con np.testing.assert_allclose

In [37]:
x = np.array([[0.8,0.7],[0.1, -0.1]])

B, z, x_recuperada = PCA_numpy(x,1)
print("PCA en Numpy")
print("Matriz de transformacion\n",B)
print("Matriz reducida\n",z)
print("Matriz recuperada\n", x_recuperada)

from sklearn.decomposition import PCA
print("\nPCA por SCIKIT")
pca = PCA(n_components=1)
pca.fit(x)
print("Matriz reducida\n",pca.transform(x))
print("Matriz recuperada\n",pca.inverse_transform(pca.transform(x)))

PCA en Numpy
Matriz de transformacion
 [[-0.65850461]
 [-0.75257669]]
Matriz reducida
 [[-0.53150729]
 [ 0.53150729]]
Matriz recuperada
 [[ 0.8  0.7]
 [ 0.1 -0.1]]

PCA por SCIKIT
Matriz reducida
 [[-0.53150729]
 [ 0.53150729]]
Matriz recuperada
 [[ 0.8  0.7]
 [ 0.1 -0.1]]
